# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [21]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

aircraft_type = 'pax'  #pax or cargo

input_path = './Data/A320_inputs.xlsx'
input_sheet = 'A320'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_A320.xlsx'

iterations = 1000000
chunks = '100000'  #'auto' or more than 500

In [22]:
from Tools import *
from Model import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [23]:
inputs = read_inputs(input_path, input_sheet)

In [24]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

In [25]:
p.data

<xarray.Dataset>
Dimensions:             (i: 1000000)
Coordinates:
  * i                   (i) int32 0 1 2 3 4 ... 999996 999997 999998 999999
Data variables: (12/81)
    takt                (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    OEW                 (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    flights_year        (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    FH                  (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    seat_max            (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    p_seat              (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    ...                  ...
    p_recycl_Al         (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    p_recycl_steel      (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    p_recycl_inconel    (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    p_recycl_Ti         (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    p_recycl_GFRP       (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>
    p_recycl_CFRP       (i) float64 dask.array<chunksize=(12500,), meta=np.ndarray>

Then, we run the function that calculates PKM for the study.

In [26]:
p = pkm(aircraft_type, p)

<br></br>
Executing the UP reading and writing functions.

In [27]:
UP_dataframe = read_unit_processes(database_path)

In [28]:
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [29]:
inventory = LCI(name=input_sheet, type=aircraft_type, iterations=iterations, UP=UP, parameters=p)

In [30]:
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 1000000)
Coordinates:
    Units           (Substances) object 'kg' 'm3' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Zirconi...
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'lower stratosphere + upper troposphe...
  * i               (i) int32 0 1 2 3 4 5 ... 999995 999996 999997 999998 999999
Data variables: (12/17)
    Office          (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Infrastructure  (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Capital         (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Materials       (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Factory         (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Logistics       (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    ...              ...
    Fuel            (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Prototypes      (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
    Certification   (Substances, i) float64 dask.array<chunksize=(1835, 12500), meta=np.ndarray>
Attributes:
    Name:     A320

The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

In [31]:
MP_data, EP_data = read_CF(database_path)

Creating an instance of the characterization factors class:

In [32]:
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [33]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [34]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object 'Carbon dioxide, fossil' ... 'Zirconi...
  - compartment     (Substances) object 'Air' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'lower stratosphere + upper troposphe...
Data variables: (12/18)
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 1.0 0.0456 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.56 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...              ...
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 1.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [35]:
aeco = LCIA.build(inventory, CF)

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [36]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 1000000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 999995 999996 999997 999998 999999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables: (12/17)
    Office          (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Infrastructure  (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Capital         (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Materials       (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Factory         (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Logistics       (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    ...              ...
    Fuel            (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Recycling       (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Incineration    (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Landfill        (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Prototypes      (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
    Certification   (i, Categories) float64 dask.array<chunksize=(12500, 18), meta=np.ndarray>
Attributes:
    Name:     A320

In [37]:
aeco.mean("MP", by='sum')

<xarray.DataArray 'stack-e980c0463b723f9b4091858d83f988e7' (Categories: 18)>
dask.array<mean_agg-aggregate, shape=(18,), dtype=float64, chunksize=(18,), chunktype=numpy.ndarray>
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [38]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 1000000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 ... 999995 999996 999997 999998 999999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/17)
    Office          (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Infrastructure  (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Capital         (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Materials       (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Factory         (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Logistics       (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    ...              ...
    Fuel            (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Recycling       (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Incineration    (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Landfill        (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Prototypes      (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
    Certification   (i, AOP) float64 dask.array<chunksize=(12500, 3), meta=np.ndarray>
Attributes:
    Name:     A320

In [42]:
aeco.mean("EP", by="sum")

<xarray.DataArray 'stack-2d1a159de84f13759c642f5f11cb6017' (AOP: 3)>
dask.array<mean_agg-aggregate, shape=(3,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [40]:
aeco.build_CTV(parameterset=p)
display(aeco.CTV)

<xarray.Dataset>
Dimensions:     (Parameters: 81)
Coordinates:
  * Parameters  (Parameters) <U18 'takt' 'OEW' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(1,), meta=np.ndarray>

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [41]:
import gc

gc.collect()

477

In [43]:
aeco.save(output_path, LCI=False)

MemoryError: Unable to allocate 175. MiB for an array with shape (1835, 12500) and data type float64

In [22]:
#aeco.to_excel(output_path)